<a href="https://colab.research.google.com/github/bennsamuel/Pneumonia-Project/blob/master/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deletes log folder
should not be used. Was implemented when cnn algorithm was not finshed to mingle around.


In [0]:

# %cd /content/drive/My Drive/Group 77 - Pneumonia/logs
# !rm -d -r *


# START HERE

In [1]:
# START HERE

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Group 77 - Pneumonia/pickle/"

data_augmented.pickle  data.pickle  test_data.pickle


In [3]:
IMG_SIZE = 224
NUM_CLASSES = 3

#                0          1         2
CATEGORIES = ["NORMAL","BACTERIA", "VIRUS"]


%cd /content/

/content


## `DATA SET which should be used for training is defined here.`

In [7]:
#load dataset --> method baseline!
import pickle

# select which pickle folder
data_set_folder_path = "/content/drive/My Drive/Group 77 - Pneumonia/pickle_augmented/"

# HERE Define here which data set should be used for training

data_set_path = "/content/drive/My Drive/Group 77 - Pneumonia/pickle_augmented/augmented-0_0_0_0_0.pickle"

# extract name from dataset
specific_data_set = data_set_path.split('/')[-1]

data = pickle.load(open(data_set_path, "rb"))

test_data = pickle.load(open("/content/drive/My Drive/Group 77 - Pneumonia/pickle/test_data.pickle", "rb"))

print("Mounted data set: {}".format(specific_data_set))

Mounted data set: augmented-0_0_0_0_0.pickle


In [0]:
# splitting data in training data and validation data 
# creating normalized SHUFFLED arrays

import numpy as np

X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

from sklearn.model_selection import train_test_split

# stratify preserves distribution of classes
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4,
                                                  random_state=42, stratify=y)

X_test = []
y_test = []

for features, label in test_data:
    X_test.append(features)
    y_test.append(label)
    
X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)


In [0]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

X_train = X_train/255.0
X_val = X_val/255.0
X_test = X_test/255.0

Create the plotting data for later. This is done at this point to have a the data before one hot encoding.

In [0]:
import pandas as pd


train_plot_data = pd.DataFrame(y_train, columns=['CLASS'])
train_plot_data['DATA SET'] = 'train'

val_plot_data = pd.DataFrame(y_val, columns=['CLASS'])
val_plot_data['DATA SET'] = 'val'

to_merge = [train_plot_data, val_plot_data]
merged_plot_data = pd.concat(to_merge)

'one hot encode' classes

before to_categorical: 

  [1, 
  2,
  1, 
  0, 
  1, 
  2, 
  ...]

after to_categorical:
  [[0,1,0],
   [0,0,1],
   [0,1,0],
   ...]

In [0]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, NUM_CLASSES)
y_val = to_categorical(y_val, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

Possibility to export target data sets.

In [0]:
# import pickle

# %cd /content/drive/My Drive/Group 77 - Pneumonia/pickle

# pickle_out = open("y_train.pickle", "wb")
# pickle.dump(y_train, pickle_out)
# pickle_out.close()

# pickle_out = open("y_val.pickle", "wb")
# pickle.dump(y_val, pickle_out)
# pickle_out.close()

# pickle_out = open("y_test.pickle", "wb")
# pickle.dump(y_test, pickle_out)
# pickle_out.close()

In [0]:
y_train.shape

Defining layers

Code partially based on [stanford cs 231](http://cs231n.github.io/convolutional-networks/)

**strides and dropout** 

Decided to use increased strides instead of maxpooling layer. Based on:
https://www.kaggle.com/dansbecker/dropout-and-strides-for-larger-models

and corresponding research paper:
["Striving for Simplicity: The All Convolutional Net"](https://academic.microsoft.com/paper/2963382180/citedby/search?q=Striving%20for%20Simplicity%3A%20The%20All%20Convolutional%20Net&qe=RId%253D2963382180&f=&orderBy=0)

Dropout percentage (0.5) based on [this paper](https://academic.microsoft.com/paper/1904365287/reference/search?q=Improving%20neural%20networks%20by%20preventing%20co-adaptation%20of%20feature%20detectors&qe=Or(Id%253D2911964244%252CId%253D2108598243%252CId%253D2112796928%252CId%253D2100495367%252CId%253D2912934387%252CId%253D2116064496%252CId%253D2147768505%252CId%253D1993882792%252CId%253D4919037%252CId%253D1498436455%252CId%253D2053229256%252CId%253D2132424367%252CId%253D2184852195%252CId%253D1567512734%252CId%253D2150884987%252CId%253D2125725328%252CId%253D2465562739)&f=&orderBy=0)

[another paper](http://jmlr.csail.mit.edu/papers/volume15/srivastava14a/srivastava14a.pdf) on dropout cited by google

Why dropout is not used on convolutional layer:
[link text](https://academic.microsoft.com/paper/2890166761/citedby/search?q=DropBlock%3A%20A%20regularization%20method%20for%20convolutional%20networks&qe=RId%253D2890166761&f=&orderBy=0)


In [0]:

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import time

NAME = "{data_set} - {time}".format(data_set = specific_data_set, time = time.strftime("%Y-%m-%d_%H:%M:%S"))

tensorboard = TensorBoard(log_dir = "/content/drive/My Drive/Group 77 - Pneumonia/logs/{}".format(NAME))

model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3),
                 strides=2,
                 activation='relu',
                 input_shape=X_train.shape[1:]))
model.add(Conv2D(64, kernel_size=(3, 3), strides=2, activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3, 3), strides=2, activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), strides=2, activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, kernel_size=(3, 3), strides=2, activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3), strides=2, activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# w/o regularization
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))

# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))

# with regularization
model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)))
model.add(Dropout(0.5))

# Dense 3 softmax because we have 3 classes
model.add(Dense(NUM_CLASSES, activation='softmax'))

Fitting the model. If this is executed multiple times make sure that no addiotional layers were added by runnning the cell above. *THIS DOES NOT WORK. YOU HAVE TO RESTART THE RUNTIME* This can be checked with the model.layers variable.

In [0]:
print("Mounted data set: {}".format(specific_data_set))

In [0]:
model.layers

In [0]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()

model.compile(loss="categorical_crossentropy", 
              optimizer="adam", 
              metrics=["accuracy", precision, recall])

model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_val,y_val), callbacks = [tensorboard])

fit model and define metrics

In [0]:
!kill 401

In [0]:
%load_ext tensorboard 

In [0]:
# has to be run twice. no clue why

%tensorboard --logdir "/content/drive/My Drive/Group 77 - Pneumonia/logs"

In [0]:
# prevents tensorboard from creating new precision_1, precision_2 etc.
tf.reset_default_graph()

# data plots

In [0]:
import plotly.express as px
fig = px.histogram(merged_plot_data, x='DATA SET', color="CLASS")
fig.show()


# Saving the model
uncomment the code below to save the model (for it to be tested on the TEST set afterwards). This **must** be the final model. 

The model is then saved in colabs storage and has to be moved to your folder of destination manually.

In [0]:
# model.save("{name}.h5".format(name="baseline"))